In [1]:
from fastai import *
from fastai.text import *
from fastai.tabular import *
from fastai import *

In [4]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [3]:
torch.backends.cudnn.enabled

True

In [3]:
TabularModel??

In [ ]:
class TabularModel(Module):
    "Basic model for tabular data."
    def __init__(self, emb_szs:ListSizes, n_cont:int, out_sz:int, layers:Collection[int], ps:Collection[float]=None,
                 emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, bn_final:bool=False):
        super().__init__()
        ps = ifnone(ps, [0]*len(layers))
        ps = listify(ps, layers)
        self.embeds = nn.ModuleList([embedding(ni, nf) for ni,nf in emb_szs])
        self.emb_drop = nn.Dropout(emb_drop)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        n_emb = sum(e.embedding_dim for e in self.embeds)
        self.n_emb,self.n_cont,self.y_range = n_emb,n_cont,y_range
        sizes = self.get_sizes(layers, out_sz)
        actns = [nn.ReLU(inplace=True) for _ in range(len(sizes)-2)] + [None]
        layers = []
        for i,(n_in,n_out,dp,act) in enumerate(zip(sizes[:-1],sizes[1:],[0.]+ps,actns)):
            layers += bn_drop_lin(n_in, n_out, bn=use_bn and i!=0, p=dp, actn=act)
        if bn_final: layers.append(nn.BatchNorm1d(sizes[-1]))
        self.layers = nn.Sequential(*layers)

    def get_sizes(self, layers, out_sz):
        return [self.n_emb + self.n_cont] + layers + [out_sz]

    def forward(self, xz_cat:Tensor, x_cont:Tensor) -> Tensor:
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)
        if self.n_cont != 0:
            x_cont = self.bn_cont(x_cont)
            x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont
        x = self.layers(x)
        if self.y_range is not None:
            x = (self.y_range[1]-self.y_range[0]) * torch.sigmoid(x) + self.y_range[0]
        return x

In [ ]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.GRU(nh, nh, 2, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(2, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [4]:
if True:
    prueba = 10

In [5]:
prueba

10

In [6]:
a = tensor([1, 2, 3])

In [8]:
b = tensor([10, 20, 30],[40, 50, 60],[70, 80, 90])

In [9]:
a

tensor([1, 2, 3])

In [10]:
b

tensor([[10, 20, 30],
        [40, 50, 60],
        [70, 80, 90]])

In [11]:
b[0] = a

In [13]:
b

tensor([[ 1,  2,  3],
        [40, 50, 60],
        [70, 80, 90]])

In [14]:
b[0] += 1

In [15]:
b[0]

tensor([2, 3, 4])

In [16]:
a

tensor([1, 2, 3])

In [17]:
a

tensor([1, 2, 3])

In [21]:
b = tensor([4, 5, 6])

In [23]:
a

tensor([1, 2, 3])

In [25]:
b

tensor([4, 5, 6])

In [28]:
x = torch.cat([a, b], 0)

In [29]:
x

tensor([1, 2, 3, 4, 5, 6])

In [30]:
x+=1

In [31]:
x

tensor([2, 3, 4, 5, 6, 7])

In [32]:
a

tensor([1, 2, 3])

In [33]:
b

tensor([4, 5, 6])